## List Tables
Exploring the tables in the 'Data Nerd Jobs' data connection.

In [ ]:
\list

## Explore 'keywords_all' Column
Analyzing the 'keywords_all' column in the 'data_nerd_jobs' table.

In [ ]:
SELECT keyword.element, COUNT(*) as count
FROM `public_job_listings.data_nerd_jobs`, UNNEST(keywords_all.list) as keyword
GROUP BY keyword.element
ORDER BY count DESC
LIMIT 10

## Plot Median Salary for Top 10 Skills
Visualizing the median salary for the top 10 skills with salary data.

In [ ]:
WITH SkillSalary AS (
  SELECT
    keyword.element AS skill,
    salary_year
  FROM
    `public_job_listings.data_nerd_jobs`,
    UNNEST(keywords_all.list) AS keyword
  WHERE
    salary_year IS NOT NULL
),
TopSkills AS (
  SELECT skill
  FROM SkillSalary
  GROUP BY skill
  ORDER BY COUNT(*) DESC
  LIMIT 10
),
MedianSalaries AS (
  SELECT
    skill,
    PERCENTILE_CONT(salary_year, 0.5) OVER (PARTITION BY skill) AS median_salary,
    COUNT(salary_year) OVER (PARTITION BY skill) AS salary_count
  FROM
    SkillSalary
  WHERE
    skill IN (SELECT skill FROM TopSkills)
)
SELECT
  DISTINCT skill,
  median_salary,
  salary_count
FROM
  MedianSalaries
ORDER BY
  median_salary DESC

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(style='darkgrid')
plt.style.use('dark_background')

# Load the dataframe
df = sql_df_fkcs

# Plotting the median salary for the top 10 skills
plt.figure(figsize=(10, 8))
salary_plot = sns.barplot(x='median_salary', y='skill', data=df, palette='Blues_r', order=df.sort_values('median_salary', ascending=False)['skill'])
plt.title('Median Salary by Skill')
plt.xlabel('Median Salary')
plt.ylabel('Skill')
plt.show()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick

sns.set(style='darkgrid')
plt.style.use('dark_background')

# Load the dataframe
df = sql_df_fkcs

# Plotting the median salary for the top 10 skills with modifications
plt.figure(figsize=(10, 8))
salary_plot = sns.barplot(x='median_salary', y='skill', data=df, palette='Blues_r', order=df.sort_values('median_salary', ascending=False)['skill'])

# Formatting the x-axis with dollar sign and comma for thousands
fmt = '${x:,.0f}'
tick = mtick.StrMethodFormatter(fmt)
salary_plot.xaxis.set_major_formatter(tick)

# Setting the x-axis limits
salary_plot.set_xlim(85000, 155000)

# Removing the axis labels
salary_plot.set_xlabel('')
salary_plot.set_ylabel('')

# Setting the title and making it bigger
salary_plot.set_title('Median Salary by Skill', fontsize=20)

# Adding the count of job postings at the end of the bars in black font and making it bigger
for p in salary_plot.patches:
    width = p.get_width()
    height = p.get_height()
    skill = df.iloc[int(p.get_y() + 0.5)]['skill']
    count = df[df['skill'] == skill]['salary_count'].iloc[0]
    x_position = width - 5000 if width > 90000 else width + 5000
    y_position = p.get_y() + height / 2
    salary_plot.annotate(f'{count} postings',
                   (x_position, y_position),
                   ha = 'center', va = 'center',
                   color = 'black',
                   size=10)

plt.show()

## Explore Salary vs Skills
Visualizing the how top skills relate to salary in a scatter plot

In [ ]:
WITH SkillSalary AS (
  SELECT
    keyword.element AS skill,
    salary_year
  FROM
    `public_job_listings.data_nerd_jobs`,
    UNNEST(keywords_all.list) AS keyword
  WHERE
    salary_year IS NOT NULL
),
TopSkills AS (
  SELECT skill
  FROM SkillSalary
  GROUP BY skill
  ORDER BY COUNT(*) DESC
--  LIMIT 10
),
MedianSalaries AS (
  SELECT
    skill,
    PERCENTILE_CONT(salary_year, 0.5) OVER (PARTITION BY skill) AS median_salary,
    COUNT(salary_year) OVER (PARTITION BY skill) AS salary_count
  FROM
    SkillSalary
  WHERE
    skill IN (SELECT skill FROM TopSkills)
)
SELECT
  DISTINCT skill,
  median_salary,
  salary_count
FROM
  MedianSalaries
ORDER BY
  median_salary DESC

In [ ]:
!pip install -q plotly
import plotly.express as px

# Load the dataframe
df = sql_df_1

# Create the scatter plot
fig = px.scatter(df, x='median_salary', y='salary_count',
                 hover_data=['skill'],
                 title='Median Salary vs. Job Posting Count',
                 labels={'median_salary': 'Median Salary', 'salary_count': 'Job Posting Count'},
                 template='plotly_dark')

# Show the plot
fig.show()

In [ ]:
WITH SkillSalary AS (
  SELECT
    keyword.element AS skill,
    job_title_final,
    salary_year
  FROM
    `public_job_listings.data_nerd_jobs`,
    UNNEST(keywords_all.list) AS keyword
  WHERE
    salary_year IS NOT NULL
),
MedianSalaries AS (
  SELECT
    skill,
    job_title_final,
    PERCENTILE_CONT(salary_year, 0.5) OVER (PARTITION BY skill, job_title_final) AS median_salary,
    COUNT(salary_year) OVER (PARTITION BY skill, job_title_final) AS salary_count
  FROM
    SkillSalary
)
SELECT
  DISTINCT skill,
  job_title_final,
  median_salary,
  salary_count
FROM
  MedianSalaries
ORDER BY
  median_salary DESC

In [ ]:
import plotly.express as px

# Load the dataframe
df = sql_df_zhgi

# Filter the dataframe to include only the three roles discussed
roles = ['Data Scientist', 'Data Analyst', 'Data Engineer']
df_filtered = df[df['job_title_final'].isin(roles)]

# Create the interactive scatter plot with a slicer for 'job_title_final'
fig = px.scatter(df_filtered, x='median_salary', y='salary_count',
                 color='job_title_final',
                 hover_data=['skill', 'job_title_final'],
                 title='Skill Count vs. Median Salary by Job Title',
                 labels={'median_salary': 'Median Salary', 'salary_count': 'Skill Count'},
                 template='plotly_dark')

# Add a dropdown to filter by job title
job_titles = df_filtered['job_title_final'].unique()
fig.update_layout(
    updatemenus=[
        dict(
            buttons=list([
                dict(args=[{'visible': [True] * len(df_filtered)}],
                     label='All',
                     method='update'),
                *[dict(args=[{'visible': [job_title==jt for job_title in df_filtered['job_title_final']]}],
                     label=jt,
                     method='update') for jt in job_titles]
            ]),
            direction='down',
            pad={'r': 10, 't': 10},
            showactive=True,
            x=0.1,
            xanchor='left',
            y=1.15,
            yanchor='top'
        ),
    ],
    showlegend=False
)

# Show the plot
fig.show()

In [ ]:
# Load the dataframe
df = sql_df_zhgi

# Normalize the 'salary_count' and 'median_salary' columns
df['normalized_count'] = (df['salary_count'] - df['salary_count'].min()) / (df['salary_count'].max() - df['salary_count'].min())
df['normalized_salary'] = (df['median_salary'] - df['median_salary'].min()) / (df['median_salary'].max() - df['median_salary'].min())

# Calculate the skill multiplier
df['skill_multiplier'] = df['normalized_count'] * df['normalized_salary']

# Display the dataframe with the new 'skill_multiplier' column
df[['skill', 'job_title_final', 'median_salary', 'salary_count', 'skill_multiplier']]

In [ ]:
import numpy as np

# Load the dataframe
df = sql_df_zhgi.copy()

# Define a function to normalize the columns within each job title group
def normalize_within_group(data, key, target):
    group_min = data.groupby(key)[target].transform('min')
    group_max = data.groupby(key)[target].transform('max')
    normalized = (data[target] - group_min) / (group_max - group_min)
    return normalized

# Apply the normalization function to the salary_count and median_salary columns
df['normalized_count'] = normalize_within_group(df, 'job_title_final', 'salary_count')
df['normalized_salary'] = normalize_within_group(df, 'job_title_final', 'median_salary')

# Calculate the skill multiplier
df['skill_multiplier'] = df['normalized_count'] * df['normalized_salary']

# Display the dataframe with the new skill_multiplier column
df[['skill', 'job_title_final', 'median_salary', 'salary_count', 'skill_multiplier', 'normalized_salary', 'normalized_count']]

# Plotting the most optimal skills
Plot the most optimal skills for data nerds based on the skill multiplier column.


In [ ]:
import plotly.graph_objs as go
from plotly.subplots import make_subplots

# Get all unique job titles
all_job_titles = df['job_title_final'].unique()

# Determine the number of rows for subplots based on the number of job titles
num_rows = len(all_job_titles)

# Create subplots for each job title with adequate spacing
fig = make_subplots(rows=num_rows, cols=1, subplot_titles=all_job_titles, vertical_spacing=0.02)

# Function to create a bar plot for top 10 skills for a given job title
def create_bar_plot(df, job_title, row):
    # Filter and sort the dataframe
    job_df = df[df['job_title_final'] == job_title].sort_values(by='skill_multiplier', ascending=False).head(10)
    # Create the bar plot
    fig.add_trace(
        go.Bar(
            x=job_df['skill'],
            y=job_df['skill_multiplier'],
            name=job_title,
            hoverinfo='text',
            hovertext=job_df.apply(lambda x: f"Skill: {x['skill']}<br>Median Salary: {x['median_salary']}<br>Salary Count: {x['salary_count']}", axis=1),
            marker=dict(color=job_df['skill_multiplier'], coloraxis='coloraxis')
        ),
        row=row,
        col=1
    )

# Add bar plots to the subplots
for i, job_title in enumerate(all_job_titles, start=1):
    create_bar_plot(df, job_title, i)

# Update layout for a cleaner look with more space between plots
fig.update_layout(
    coloraxis=dict(colorscale='Bluered_r'),
    showlegend=False,
    title_text='Optimal Skills Across Data Roles',
    title_x=0.5,
    height=300 * num_rows,  # Adjust the height based on the number of rows
    xaxis_tickangle=-45
)

# Show the plot
fig.show()

In [ ]:
import plotly.graph_objs as go
from plotly.subplots import make_subplots

# Define the job titles to include
included_job_titles = ['Data Analyst', 'Data Engineer', 'Data Scientist']

# Filter the dataframe for the included job titles
filtered_df = df[df['job_title_final'].isin(included_job_titles)]

# Determine the number of rows for subplots based on the number of included job titles
num_rows = len(included_job_titles)

# Create subplots for each job title with adequate spacing
fig = make_subplots(rows=num_rows, cols=1, subplot_titles=included_job_titles, vertical_spacing=0.1)

# Function to create a bar plot for top 10 skills for a given job title
def create_bar_plot(df, job_title, row):
    # Filter and sort the dataframe
    job_df = df[df['job_title_final'] == job_title].sort_values(by='skill_multiplier', ascending=False).head(10)
    # Create the bar plot
    fig.add_trace(
        go.Bar(
            x=job_df['skill'],
            y=job_df['skill_multiplier'],
            name=job_title,
            hoverinfo='text',
            hovertext=job_df.apply(lambda x: f"Skill: {x['skill']}<br>Median Salary: {x['median_salary']}<br>Salary Count: {x['salary_count']}", axis=1),
            marker=dict(color=job_df['skill_multiplier'], coloraxis='coloraxis')
        ),
        row=row,
        col=1
    )

# Add bar plots to the subplots
for i, job_title in enumerate(included_job_titles, start=1):
    create_bar_plot(filtered_df, job_title, i)

# Update layout for a cleaner look with more space between plots
fig.update_layout(
    coloraxis=dict(colorscale='Bluered_r'),
    showlegend=False,
    title_text='Optimal Skills for Data Nerds',
    title_x=0.5,
    height=350 * num_rows,  # Adjust the height based on the number of rows
    xaxis_tickangle=-45
)

# Show the plot
fig.show()